In [1]:
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ

os.environ["DIPY_HOME"] = "/home/jovyan/Data"


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json
import subprocess

Gtk-Message: 15:33:31.475: Failed to load module "canberra-gtk-module"


In [2]:
################
# Start FSLeyes (very neat tool to visualize MRI data of all sorts) within Python
################
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

15:33:43: Debug: Adding duplicate image handler for 'Windows bitmap file'
15:33:43: Debug: Adding duplicate animation handler for '1' type
15:33:43: Debug: Adding duplicate animation handler for '2' type
15:33:43: Debug: Adding duplicate image handler for 'Windows bitmap file'
15:33:43: Debug: Adding duplicate animation handler for '1' type
15:33:43: Debug: Adding duplicate animation handler for '2' type

(ipykernel_launcher.py:565): Gtk-CRITICAL **: 15:33:44.141: gtk_window_resize: assertion 'height > 0' failed


In [8]:
# Variables
dataset_id = "ds000171"
bids_root = "./ds000171"  # Directory to store the downloaded dataset
# Path to the 'derivatives' directory for processed data
deriv_root = os.path.join(bids_root, 'derivatives')

# Create a directory for preprocessed data (e.g., 'preprocessed_data')
preproc_root = os.path.join(deriv_root, 'preprocessed_data')

In [9]:


# Create the target directory if it does not exist
if not os.path.exists(bids_root):
    os.makedirs(bids_root)

# Download the raw data from OpenNeuro into the target directory
#openneuro.download(dataset=dataset_id, target_dir= bids_root)




In [10]:
if not os.path.exists(preproc_root):
    os.makedirs(preproc_root)

In [11]:
subjects = [d for d in os.listdir(bids_root) if d.startswith('sub-con')]
# Loop over all subjects and create directories for each subject's processed data
for subject in subjects:
    subject_dir = os.path.join(preproc_root, subject)

    # Create subdirectories for anatomical and functional data
    os.makedirs(os.path.join(subject_dir, 'anat'), exist_ok=True)
    os.makedirs(os.path.join(subject_dir, 'func'), exist_ok=True)

print(f"Directories created for processed data in {preproc_root}")

Directories created for processed data in ./ds000171/derivatives/preprocessed_data


In [12]:

subjects = [d for d in os.listdir(bids_root) if d.startswith('sub-con')]
anatomical_path = op.join(bids_root, subjects[0], 'anat', '{}_T1w.nii.gz'.format(subjects[0]))

# Check if the anatomical file exists before calling BET
if not os.path.exists(anatomical_path):
    print(f"Error: Anatomical file not found: {anatomical_path}")
else : 
    print("oui")

oui


In [13]:
#________________________________ ANATOMICAL PREPROCESSING ____________________________________________
subjects = [d for d in os.listdir(bids_root) if d.startswith('sub-con')]
def get_skull_stripped_anatomical(bids_root, preproc_root, subject, robust=False):
    """
    Function to perform skull-stripping (removing the skull around the brain).
    This is a simple wrapper around the brain extraction tool (BET) in FSL's suite
    It assumes data to be in the BIDS format (which we will cover in the following weeks).
    The method also saves the brain mask which was used to extract the brain.

    The brain extraction is conducted only on the T1w of the participant.

    Parameters
    ----------
    bids_root: string
        The root of the BIDS directory
    preproc_root: string
        The root of the preprocessed data, where the result of the brain extraction will be saved.
    subject_id: string
        Subject ID, the subject on which brain extraction should be conducted.
    robust: bool
        Whether to conduct robust center estimation with BET or not. Default is False.
    """
    anatomical_path = op.join(bids_root, subject, 'anat', '{}_T1w.nii.gz'.format(subject))
    betted_brain_path = op.join(preproc_root, subject, 'anat', '{}_T1w'.format(subject))
    os.system('bet {} {} -m {}'.format(anatomical_path, betted_brain_path, '-R' if robust else ''))




for sub in tqdm(subjects) : 
    get_skull_stripped_anatomical(bids_root, preproc_root, sub)
    print("Done with BET.")


  5% 1/20 [00:04<01:25,  4.52s/it]

Done with BET.


 10% 2/20 [00:07<01:02,  3.47s/it]

Done with BET.


 15% 3/20 [00:09<00:53,  3.12s/it]

...

Done with BET.


 95% 19/20 [00:58<00:02,  2.99s/it]

Done with BET.


100% 20/20 [01:01<00:00,  3.07s/it]

Done with BET.


In [15]:
# Pour check que bien marcher : 
resulting_mask_path = op.join(preproc_root, subject, 'anat', '{}_T1w_mask'.format(subjects[1]))
fsleyesDisplay.load(resulting_mask_path)

PathError: Could not find a supported file with prefix "/home/jovyan/ds000171/derivatives/preprocessed_data/sub-control14/anat/sub-control07_T1w_mask"